# Imports

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#device = 0
#torch.cuda.set_device(device)

%reload_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline
from bokeh.io import output_notebook


import sys

#https://github.com/FAU-DLM/python-wsi-preprocessing
sys.path.append('../preprocessing_pipeline/python-wsi-preprocessing/')
import wsi
from wsi import tiles, util

sys.path.append('../fastai/')
import fastai
from fastai.vision import *
from fastai.vision.learner import model_meta
from fastai.callbacks import *

sys.path.append('../models-pytorch/pretrained-models.pytorch')
import pretrainedmodels
from pretrainedmodels import *

from typing import Dict
import pandas as pd
import numpy as np
import os
import torch
import torchvision
from torchvision.models import *
from torchsummary import summary
from pathlib import Path
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import shutil
from sklearn.model_selection import StratifiedKFold, KFold
from typing import List, Callable
from PIL import Image
Image.MAX_IMAGE_PIXELS = 10000000000   

PATH = Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/')

WSIS_CORTICOTROP = PATH/'corticotrop'
WSIS_GONADOTROP = PATH/'gonadotrop'

ROIS_CORTICOTROP = PATH/'corticotrop_ROIs'
ROIS_GONADOTROP = PATH/'gonadotrop_ROIs'

TILES_CORTICOTROP_1 = PATH/'tiles_corticotrop_1_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_2 = PATH/'tiles_corticotrop_2_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_3 = PATH/'tiles_corticotrop_3_scoring_function_1_thresh_0.4'

TILES_GONADOTROP_1 = PATH/'tiles_gonadotrop_1_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_2 = PATH/'tiles_gonadotrop_2_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_3 = PATH/'tiles_gonadotrop_3_scoring_function_1_thresh_0.4'

#TEST = PATH/TEST_NAME
#TEST = PATH_LOCAL/TEST_NAME
TEST_EXPERIMENTING = PATH/'tiles_test_100_for_testing'
LABELS_CORTICOTROP_NAME = 'KortikotropHA_gelabled.xlsx'
LABELS_CORTICOTROP = PATH/LABELS_CORTICOTROP_NAME
LABELS_GONADOTROP_NAME = 'GonadotropeHA_gelabled.xlsx'
LABELS_GONADOTROP = PATH/LABELS_GONADOTROP_NAME
MODEL_PATH_NAME = 'models'
MODEL_PATH = PATH/MODEL_PATH_NAME

ROIS_EXPERIMENTING = PATH/'rois_experimenting'
TILES_EXPERIMENTING = PATH/'tiles_experimenting'

torch.backends.cudnn.benchmark=True

#def batch_stats(self, funcs:Collection[Callable]=None)->Tensor:
#        "Grab a batch of data and call reduction function `func` per channel"
#        funcs = ifnone(funcs, [torch.mean,torch.std])
#        x = self.one_batch(ds_type=DatasetType.Train, denorm=False)[0].cpu()
#        return [func(channel_view(x), 1) for func in funcs]
#        
#vision.data.ImageDataBunch.batch_stats = batch_stats

sz = 512
bs = 6

#fastai defaults
tta_beta = 0.4 
tta_scale = 1.35
dropout = 0.5
wd = 0.01

#non defaults
#wd = 0.1 not better for se_resnext50
#dropout = 0.9


seed = 19
np.random.seed(seed)

num2lbs = {
    0:"corticotrop", 
    3:"silent",  
    8:"LH", 
    9:"FSH"
}

lbs2num = {l:n for n,l in num2lbs.items()}

# Utils

In [ ]:
from fastai.torch_core import flatten_model

def arch_summary(arch):
    model = arch(False)
    tot = 0
    for i, l in enumerate(model.children()):
        n_layers = len(flatten_model(l))
        tot += n_layers
        print(f'({i}) {l.__class__.__name__:<12}: {n_layers:<4}layers (total: {tot})')

def show(np):
    return util.np_to_pil(np)

Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

def show_multiple_images(path, rows = 3, figsize=(128, 64)):
    imgs = [open_image(p) for p in path.ls()]
    show_all(imgs=imgs, r=rows, figsize=figsize)
    
def show_multiple_images_big(path:pathlib.Path):
    for p in path.ls():
        plt.imshow(mpimg.imread(str(p)))
        plt.show()
        
def get_id_from_path(path):
    path = Path(path)
    split = path.stem.split('-')
    return f'{split[0]}-{split[1]}'

def get_slide_name_from_path(path):
    path = Path(path)
    split = path.stem.split('-')
    try:
        return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}'
    except IndexError:
        return f'{split[0]}-{split[1]}-{split[2]}'
    
def extract_case_ids_from_existing_tiles()->tuple((List[str], List[pathlib.Path])):
    tile_paths_gonadotrop_1 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_GONADOTROP_1.ls()) if p.suffix == '.png']
    tile_paths_gonadotrop_2 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_GONADOTROP_2.ls()) if p.suffix == '.png']
    tile_paths_gonadotrop_3 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_GONADOTROP_3.ls()) if p.suffix == '.png']
    
    tile_paths_corticotrop_1 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_CORTICOTROP_1.ls()) if p.suffix == '.png']
    tile_paths_corticotrop_2 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_CORTICOTROP_2.ls()) if p.suffix == '.png']
    tile_paths_corticotrop_3 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_CORTICOTROP_3.ls()) if p.suffix == '.png']
    
    tile_paths_all = tile_paths_gonadotrop_1 \
                        + tile_paths_gonadotrop_2 \
                        + tile_paths_gonadotrop_3 \
                        + tile_paths_corticotrop_1 \
                        + tile_paths_corticotrop_2 \
                        + tile_paths_corticotrop_3 \
    
    return list(set([get_id_from_path(p) for p in tile_paths_all])), tile_paths_all


df_c = pd.read_excel(LABELS_CORTICOTROP)
def label_func(path):
    path = Path(path)
    s = path.stem  
    if('LH+FSH' in s):
        return [lbs2num['LH'],lbs2num['FSH']]
    elif 'LH' in s:       
        return [lbs2num['LH']]
    elif 'FSH' in s:        
        return [lbs2num['FSH']]
    elif 'ACTH' in s:
        result = [lbs2num['corticotrop']]
        id = get_id_from_path(path)
        l = df_c.loc[df_c.id == id].label
        try:
            if str(lbs2num['silent']) in str(l.values[0]):
                result.append(3)
        except:
            print(l.values)
            print(s)
            print(get_id_from_path(path))
            raise
        return result

# Extra Models

In [ ]:
#https://github.com/PPPW/deep-learning-random-explore/blob/master/CNN_archs/cnn_archs.ipynb

def identity(x): return x

def nasnetamobile(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.nasnetamobile(pretrained=pretrained, num_classes=1000)  
    model.logits = identity
    model_meta[nasnetamobile] =  { 'cut': identity, 'split': lambda m: (list(m[0][0].children())[8], m[1]) }
    return nn.Sequential(model)

#arch_summary(lambda _: nasnetamobile(False)[0])

def se_resnext50_32x4d(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext50_32x4d(pretrained=pretrained)
    model_meta[se_resnext50_32x4d] =  {'cut': -2, 'split': lambda m: (m[0][3], m[1]) }
    return model

#arch_summary(lambda _: pretrainedmodels.se_resnext50_32x4d(pretrained=None))

def se_resnext101_32x4d(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext101_32x4d(pretrained=pretrained)
    model_meta[se_resnext101_32x4d] =  {'cut': -2, 'split': lambda m: (m[0][3], m[1]) }
    return model

def xception(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.xception(pretrained=pretrained)
    model_meta[xception] =  { 'cut': -1, 'split': lambda m: (m[0][11], m[1]) }
    return model

def inceptionv4(pretrained=True):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.inceptionv4(pretrained=pretrained)
    model_meta[xception] =  { 'cut': -2, 'split': lambda m: (m[0][11], m[1]) }
    return model

# n 

In [ ]:
#n='test'

n = np.load('n.npy')
print(n)

m = n+1
m=13
np.save('n', m)
print(m)

# Data 

## Some numbers of the dataset (not necessary for training)

### class distribution

In [ ]:
### check for possible mistakes in label function
for tile_p in tile_paths_all:
    lbl = label_func(tile_p)
    if (0 in lbl or 3 in lbl) and (8 in lbl or 9 in lbl):
        print(tile_p)
        print(lbl)

In [ ]:
acth_tiles = 0
silent_acth_tiles = 0
lh_tiles = 0
fsh_tiles = 0
gonadotropic_tiles = 0
acth_cases = 0
silent_acth_cases = 0
lh_cases = 0
fsh_cases = 0
gonadotropic_cases = 0

case_ids_visited = []

for tile_path in tile_paths_all:
    lbl = label_func(tile_path)
    case_id = get_id_from_path(tile_path)
    if 0 in lbl:
        acth_tiles += 1
        if case_id not in case_ids_visited:
            acth_cases += 1
    if 3 in lbl:
        silent_acth_tiles += 1
        if case_id not in case_ids_visited:
            silent_acth_cases += 1
    if 8 in lbl:
        lh_tiles += 1
        if case_id not in case_ids_visited:
            lh_cases += 1
    if 9 in lbl:
        fsh_tiles += 1
        if case_id not in case_ids_visited:
            fsh_cases += 1
    if 8 in lbl or 9 in lbl:
        gonadotropic_tiles += 1
        if case_id not in case_ids_visited:
            gonadotropic_cases += 1
    if case_id not in case_ids_visited:
        case_ids_visited.append(case_id)

In [ ]:
len(case_ids_visited)

In [ ]:
len(case_ids)

In [ ]:
print(f'tiles with label "acth": {acth_tiles/len(tile_paths_all)*100}%')
print(f'tiles with label "silent_acth": {silent_acth_tiles/len(tile_paths_all)*100}%')
print(f'tiles with label "lh": {lh_tiles/len(tile_paths_all)*100}%')
print(f'tiles with label "fsh": {fsh_tiles/len(tile_paths_all)*100}%')
print(f'tiles with label "lh or fsh": {gonadotropic_tiles/len(tile_paths_all)*100}%')
print(f'cases with label "acth": {acth_cases/len(case_ids)*100}%')
print(f'cases with label "silent_acth": {silent_acth_cases/len(case_ids)*100}%')
print(f'cases with label "lh": {lh_cases/len(case_ids)*100}%')
print(f'cases with label "fsh": {fsh_cases/len(case_ids)*100}%')
print(f'cases with label "lh or fsh": {gonadotropic_cases/len(case_ids)*100}%')

### gonadotropic

In [ ]:
# H&E and immunos
all_wsis_paths_gonado = [p for p in WSIS_GONADOTROP.ls() if p.suffix == '.ndpi']; len(all_wsis_paths_gonado)

In [ ]:
# all gonadotropic HE WSIs

wsi_names_gon = set([p for p in WSIS_GONADOTROP.ls() 
                     if ('HE' in str(p) \
                     and not ('LH' in str(p) or 'FSH' in str(p)) \
                     and p.suffix == '.ndpi')])
print(len(wsi_names_gon))

In [ ]:
# all gonadotropic cases == number of patients (one case per patient)
len(set([get_id_from_path(p) for p in WSIS_GONADOTROP.ls()]))

In [ ]:
# number of cases, ROIs have been extracted from
len(set([get_id_from_path(p) for p in ROIS_GONADOTROP.ls()]))

In [ ]:
# number of ROIs
len(ROIS_GONADOTROP.ls())

In [ ]:
tile_paths_gonadotrop_1 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_GONADOTROP_1.ls()) if p.suffix == '.png']
tile_paths_gonadotrop_2 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_GONADOTROP_2.ls()) if p.suffix == '.png']
tile_paths_gonadotrop_3 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_GONADOTROP_3.ls()) if p.suffix == '.png']
tile_paths_all_gonadotrop = tile_paths_gonadotrop_1 \
                            + tile_paths_gonadotrop_2 \
                            + tile_paths_gonadotrop_3 

In [ ]:
# number of cases, tiles have been extracted from
len(set([get_id_from_path(p) for p in tile_paths_all_gonadotrop]))

In [ ]:
len(tile_paths_all_gonadotrop)

### corticotropic

In [ ]:
# H&E and immunos
all_wsis_paths_cortico = [p for p in WSIS_CORTICOTROP.ls() if p.suffix == '.ndpi']; len(all_wsis_paths_cortico)

In [ ]:
# all corticotropic HE WSIs
wsi_names_cort = set([p for p in WSIS_CORTICOTROP.ls() if ('HE' in str(p) and not 'ACTH' in str(p) and p.suffix == '.ndpi')])
print(len(wsi_names_cort))

In [ ]:
# all corticotropic cases == number of patients (one case per patient, but some cases have more than one HE WSI)
len(set([get_id_from_path(p) for p in WSIS_CORTICOTROP.ls()]))

In [ ]:
# number of cases, ROIs have been extracted from
len(set([get_id_from_path(p) for p in ROIS_CORTICOTROP.ls()]))

In [ ]:
# number of ROIs
len(ROIS_CORTICOTROP.ls())

In [ ]:
tile_paths_corticotrop_1 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_CORTICOTROP_1.ls()) if p.suffix == '.png']
tile_paths_corticotrop_2 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_CORTICOTROP_2.ls()) if p.suffix == '.png']
tile_paths_corticotrop_3 = [Path(f'{p.parts[-2]}/{p.parts[-1]}') for p in (TILES_CORTICOTROP_3.ls()) if p.suffix == '.png']


tile_paths_all_corticotrop = tile_paths_corticotrop_1 \
                            + tile_paths_corticotrop_2 \
                            + tile_paths_corticotrop_3 \

In [ ]:
# number of cases, tiles have been extracted from
len(set([get_id_from_path(p) for p in tile_paths_all_corticotrop]))

In [ ]:
len(tile_paths_all_corticotrop)

## Create pandas dataframe with tile information to later extract tiles on the fly from WSIs during training (use this, if you do not have extracted tiles saved on disc)

In [ ]:
tiles_df_path = PATH/'tiles_info-tile_score_thresh=0.4-tiles.scoring_function_1.csv'


if os.path.isfile(tiles_df_path):
    ###
    # just load from disc, if you have already calculated tile infos before
    ###
    tiles_df = pd.read_csv(tiles_df_path).set_index('tile_name')
else:
    ###
    # generate and save tile info
    ###
    rois_paths_gonado = [p for p in ROIS_GONADOTROP.ls() if (p.suffix == '.png' and '-HE' in p.name)]
    rois_paths_cortico = [p for p in ROIS_GONADOTROP.ls() if (p.suffix == '.png' and '-HE' in p.name)]
    rois_paths_all = rois_paths_gonado + rois_paths_cortico
    tiles_df = tiles.WsiOrROIToTilesMultithreaded(wsiPaths=rois_paths_all, 
                                   tilesFolderPath=None, 
                                   tileHeight=1024, 
                                   tileWidth=1024, 
                                   tile_naming_func=tiles.get_roi_name_from_path_pituitary_adenoma_entities, 
                                   tile_score_thresh=0.4, 
                                   tileScoringFunction=tiles.scoring_function_1, 
                                   is_wsi=False, 
                                   level=0, 
                                   save_tiles=False)
    tiles_df.to_csv(tiles_df_path, index_label='tile_name')    

## split dataset

### moving tiles into seperate folders -- obsolete

In [ ]:
##
#specify test data and move it to a seperate folder (required only once)
##
#tile_paths_all = [p for p in (TRAIN.ls()) if p.suffix == '.png']
#ids = []
#for p in tqdm(tile_paths_all):
#    ids.append(get_id_from_path(p))
#ids = list(set(ids))
#train_and_valid_pct = 0.9
#test_pct = 0.1
#ids_train_and_valid, ids_test = train_test_split(ids, test_size=test_pct, random_state=seed)
#
###
##move test images to extra folder
###
#for id in tqdm(ids_test):
#    for p in tile_paths_all:
#        if id in str(p):
#            !mv {p} {TEST}

##
#split remaining images into train and val sets
##
#tile_paths_train_and_valid = [p for p in (TRAIN.ls()) if p.suffix == '.png']
#ids_train_and_val = []
#for p in tqdm(tile_paths_train_and_valid):
#    ids_train_and_val.append(get_id_from_path(p))       
#ids_train_and_val = list(set(ids_train_and_val))
#train_pct = 0.8
#valid_pct = 0.2
#ids_train, ids_val = train_test_split(ids_train_and_val, test_size=valid_pct, random_state=seed)

### create unique list of case ids and list of all tiles

In [ ]:
###
# Option 1: use this, if you already have extracted tiles saved to disc
###
case_ids, tile_paths_all = extract_case_ids_from_existing_tiles()

###
# Option 2: use this, if you only have a dataframe generated in 5.2
###
#case_ids = list(set([get_id_from_path(p) for p in tiles_df['wsi_path'].tolist()]))
#tile_paths_all = tiles_df.index.to_list()

### Option 1: split cases and the associated tiles into train, valid and test set

In [ ]:
train_and_valid_pct = 0.9
test_pct = 0.1
ids_train_and_valid, ids_test = train_test_split(case_ids, test_size=test_pct, random_state=seed)

valid_pct = 0.2
ids_train, ids_val = train_test_split(ids_train_and_valid, test_size=valid_pct, random_state=seed)

tile_paths_train = [p for p in tile_paths_all if get_id_from_path(p) in ids_train]
tile_paths_val = [p for p in tile_paths_all if get_id_from_path(p) in ids_val]
tile_paths_test = [p for p in tile_paths_all if get_id_from_path(p) in ids_test]

df_tile_paths_train_and_valid = pd.DataFrame((tile_paths_train+tile_paths_val), columns=['name'])

print(f'seed: {seed}')
print(len(tile_paths_train))
print(len(tile_paths_val))
print(len(tile_paths_test))
print(len(ids_train))
print(len(ids_val))
print(len(ids_test))
print(len(ids_train+ids_val+ids_test))

### Option 2: n-fold cross validation

In [ ]:
iteration = 2
n_splits = 5

###
# Also generating y-labels for stratified splits, but that's very hard to decide for a multilabel classification
# (sklearn.model_selection.StratifiedKFold)
###

##key = tile_path:string; value = labels:list[ints]
#tiles_paths_to_labels = {}
#for p in tile_paths_all:
#    lb = label_func(p)
#    assert not((1 in lb or 3 in lb) and (8 in lb or 9 in lb)) and len(lb) < 3
#    tiles_paths_to_labels[p] = lb
#
##key = id:string; value = labels:list[ints]
#case_id_to_labels = {}
#for p in tiles_paths_to_labels:
#    lb_tile = tiles_paths_to_labels[p]
#    case_id = get_id_from_path(p)   
#    if(case_id in case_id_to_labels):
#        case_id_to_labels[case_id] += lb_tile
#    else:
#        case_id_to_labels[case_id] = lb_tile
#    lb_case_id = case_id_to_labels[case_id]
#    assert not((1 in lb_case_id or 3 in lb_case_id) and (8 in lb_case_id or 9 in lb_case_id)) and len(lb_case_id) < 3   
#
#x_case_id_indices = list(range(len(case_id_to_labels)));x_case_id_indices    
#
#def one_hot_encode(labels:list, all_classes:list = lbs2num.values()):
#    for c in labels:
#        assert c in all_classes
#    n = len(all_classes)
#    res = np.zeros(n, int)
#    for i, c in enumerate(all_classes):
#        if c in labels:
#            res[i] = 1 
#    return res
#
#y = np.zeros(shape=(len(case_id_to_labels),len(lbs2num.values())))
#for n, case_id in enumerate(case_ids):
#    y[n] = one_hot_encode(case_id_to_labels[case_id])


x_case_id_indices = list(range(len(case_ids)))

kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

splits = kf.split(x_case_id_indices)
split_current_iteration = list(splits)[iteration]

train_indices = split_current_iteration[0]
val_indices = split_current_iteration[1]

ids_train = [case_ids[i] for i in train_indices]
ids_val = [case_ids[i] for i in val_indices]

df_tile_paths_train_and_valid = pd.DataFrame(tile_paths_all, columns=['name'])

## Transforms

In [ ]:
tfms = get_transforms(flip_vert=True)

#for t in tfms[0]:
#    print(t)
#    print("--------------------------------------------------------------------------------")
#    
#for t in tfms[1]:
#    print(t)
#    print("--------------------------------------------------------------------------------")

#tfms = ([RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True),
#        RandTransform(tfm=TfmLighting (brightness), kwargs={'change': (0.475, 0.525)}, p=0.75, resolved={}, do_run=True, is_random=True),
#        RandTransform(tfm=TfmLighting (contrast), kwargs={'scale': (0.95, 1.0526315789473684)}, p=0.75, resolved={}, do_run=True, is_random=True)],
#        [])

#def get_ex(): return open_image(str(TRAIN.ls()[0]))
#
#def plots_f(rows, cols, width, height, **kwargs):
#    [get_ex().apply_tfms(tfms[0], **kwargs).show(ax=ax) for i,ax in enumerate(plt.subplots(
#        rows,cols,figsize=(width,height))[1].flatten())]
#
#plots_f(2, 4, 12, 6, size=224)

## Datablock API

In [ ]:
###
# if you use a pandas dataframe generated in 5.2 to extract tiles on the fly during training, 
# overwrite fastai.vision.data.ImageList.open and fastai.vision.image.open_image
###

#here we already have extracted tiles and would need to extract from ROIs (large .png file), which is pretty slow.
#for a working example, where tiles get extracted from WSIs on the fly, see the network-relapse_classification.ipynb

#def open_custom(self, fn):
#    "Open image in `fn`."
#    return open_image_custom(fn, convert_mode=self.convert_mode, after_open=self.after_open)
#
#def open_image_custom(fn:PathOrStr, 
#                      div:bool=True, 
#                      convert_mode:str='RGB', 
#                      cls:type=fastai.vision.Image, 
#                      after_open:Callable=None)->fastai.vision.Image:
#        "Open image in `fn`."
#        fn = Path(fn)
#        tile_name = fn.name
#        row = tiles_df.loc[tile_name, : ]
#        wsi_path = row['wsi_path']
#        x = row['x_upper_left']
#        y = row['y_upper_left']
#        width = row['pixels_width']
#        height = row['pixels_height']
#        tile = tiles.ExtractTileFromPILImage(path=wsi_path, x=x, y=y, width=width, height=height)
#        tile = tile.convert(convert_mode)
#        if after_open: 
#            tile = after_open(tile)
#        tile = pil2tensor(tile,np.float32)
#        if div: 
#            tile.div_(255)
#        return cls(tile)
#        
#fastai.vision.data.ImageList.open = open_custom
#fastai.vision.image.open_image = open_image_custom

nw = 16   #number of workers for data loader

def split_func(path):
    path = Path(path)
    return get_id_from_path(path) in ids_val

#data = ImageList.from_folder(path=TRAIN, extensions=['.png'])
data = ImageList.from_df(df_tile_paths_train_and_valid, path=PATH)
data = data.split_by_valid_func(split_func)
data = data.label_from_func(label_func)
data = data.transform(tfms=tfms, size=sz)
#data = data.add_test_folder(test_folder=TEST_EXPERIMENTING)
#data = data.add_test([PATH/p for p in tile_paths_test])
temporary_training_path = PATH/f'models/{n}-resnext_currently_training_cross-valid-iteration-{iteration}'
data = data.databunch(bs=bs, num_workers=nw, path=temporary_training_path)
data = data.normalize()

# Learner

## Create

In [ ]:
epochs_frozen = 5
epochs_unfrozen = 10

In [ ]:
arch = resnext101_32x8d
learner = cnn_learner(data=data, 
                     base_arch=arch, 
                     metrics=[accuracy_thresh], 
                     ps=dropout, 
                     pretrained=True, 
                     wd = wd)

## Name

In [ ]:
#nameBase = f'{n}-{arch.__name__}-size{sz}-bs{bs}-epochs_head{epochs_frozen}-epochs_complete{epochs_unfrozen}-seed_{seed}-test_pct_{test_pct}-valid_pct_{valid_pct}-with_tiles_gonadotrop2_and_corticotrop2'
valid_pct = len(ids_val)/len(case_ids)
nameBase = f'{n}-{arch.__name__}-size{sz}-bs{bs}-epochs_head{epochs_frozen}-epochs_complete{epochs_unfrozen}-seed_{seed}-valid_pct_{valid_pct}-n_splits_{n_splits}-tiles_1+2+3-cross_valid_iteration_{iteration}'
nameBase

## Train

In [ ]:
learner.lr_find(start_lr=1e-10, end_lr=10, num_it=1000)

In [ ]:
plt.rcParams.update({'font.size': 15})

In [ ]:
learner.recorder.plot()

In [ ]:
lr = 1e-3

In [ ]:
learner.fit_one_cycle(cyc_len=epochs_frozen, max_lr=lr, 
                      callbacks=[SaveModelCallback(learner, 
                                                   every='epoch', monitor='accuracy_thresh', 
                                                   name='bestmodel_head')])

In [ ]:
nameHead = f'{nameBase}-head'

In [ ]:
nameHead

In [ ]:
learner.save(nameHead)

In [ ]:
#learner.load(nameHead)
learner.load('bestmodel_head_4')

In [ ]:
learner.unfreeze()

In [ ]:
learner.lr_find(start_lr=1e-12, end_lr=10, num_it=10000)
learner.recorder.plot(skip_start=0)

In [ ]:
lr2 = 1e-7
lr3 = 1e-6

In [ ]:
learner.fit_one_cycle(cyc_len=epochs_unfrozen, 
                      max_lr=slice(lr2, lr3), 
                      callbacks=[SaveModelCallback(learner, 
                                                   every='epoch', 
                                                   monitor='accuracy_thresh', 
                                                   name='bestmodel_lr2=1e-7--lr3=1e-6')])

In [ ]:
learner.recorder.plot_losses()

In [ ]:
learner.recorder.plot_lr()

In [ ]:
learner.recorder.plot_metrics()

In [ ]:
nameComplete = f'{nameBase}-complete'

In [ ]:
#learner.save(nameComplete)

In [ ]:
learner.load(nameComplete)

In [ ]:
#learner.load('bestmodel_9')

# Prediction per case

In [ ]:
def one_hot_encode(predicted_classes:list, all_classes:list):
    for c in predicted_classes:
        assert c in all_classes
    n = len(all_classes)
    res = np.zeros(n, int)
    for i, c in enumerate(all_classes):
        if c in predicted_classes:
            res[i] = 1 
    return res



def ensemble_predict(dict_arch_to_path_of_saved_model:typing.Dict[Callable, pathlib.Path], 
                     data:fastai.vision.data.ImageDataBunch,
                     ds_type:fastai.basic_data.DatasetType,
                     tta:bool, 
                     scale:float,
                     beta:float):
    """
    tta: Should test time augmentation be used?
    scale: if tta is True -> scaling factor for tta
    beta: if tta is True -> beta factor for tta
    check this out for more infos: https://docs.fast.ai/basic_train.html#Test-time-augmentation
    """
   
    print(f'{str([a.__name__ for a in dict_arch_to_path_of_saved_model.keys()])}_sz{sz}_ensembled')
    
    predsList = []
    for arch in dict_arch_to_path_of_saved_model.keys():
        learner = cnn_learner(data=data, base_arch=arch, pretrained=False)
        learner.load(dict_arch_to_path_of_saved_model[arch])
        if tta is True:
            preds = learner.TTA(beta=beta, scale=scale, ds_type=ds_type)
        else:
            preds = learner.get_preds(ds_type=ds_type)
            
        predsList.append(preds)
    
    preds_ensembled = predsList[0]
    for n, _ in enumerate(predsList):
        if n == 0:
            continue
        else:
            preds_ensembled[0] = preds_ensembled[0] + predsList[n][0]
    preds_ensembled[0] = preds_ensembled[0]/len(predsList)
    
    return preds_ensembled

def from_preds_to_dict_path_to_preds(preds, 
                                     imageDataBunch:fastai.vision.ImageDataBunch, 
                                     ds_type:fastai.basic_data.DatasetType,
                                     threshold:float):
    """
    preds: What fastai.vision.learner.get_preds or fastai.vision.learner.TTA return.
            two tensors: 1st: lists with raw predictions for each class of an image
                         2nd: lists with y_true
            form e.g. [tensor([[0.9672, 0.9211, 0.4560, 0.8185], 
                                [0.9498, 0.8600, 0.5852, 0.7206]]),
                         tensor([[0., 0., 0., 1.],
                                [0., 0., 1., 1.]])]
    threshold:  threshold to consider the predictions per tile to be correct or not
                                
    RETURN:
        key:path, value:tuple (fastai.core.MultiCategory, tensor preds one hot encoded, tensor with pure preds) 
        e.g. (MultiCategory 0, tensor([1., 0., 0., 0.]), tensor([0.9952, 0.0015, 0.0021, 0.0029]))
    path_to_pred = {}
    """
    #key:path, value:tuple (fastai.core.MultiCategory, tensor preds one hot encoded, tensor with pure preds) 
    #e.g. (MultiCategory 0, tensor([1., 0., 0., 0.]), tensor([0.9952, 0.0015, 0.0021, 0.0029]))
    path_to_pred = {}
    d = None
    if ds_type is DatasetType.Valid:
        d = imageDataBunch.valid_ds
    elif ds_type is DatasetType.Test:
        d = imageDataBunch.test_ds
    elif ds_type is DatasetType.Train:
        d = imageDataBunch.train_ds
    for path, pred in tqdm(zip(d.items, preds[0]), total = len(d.items)):
        multi_c = None
        pred_one_hot_encoded = (pred > threshold).float()
        pred_raw = pred
        path_to_pred[path] = multi_c, pred_one_hot_encoded, pred_raw
        
    return path_to_pred


def get_class_occurence_per_id(learner:fastai.vision.learner=None,
                               labelList:fastai.data_block.LabelList=None,
                               dict_arch_to_path_of_saved_model:typing.Dict[Callable, pathlib.Path]=None,
                               imageDataBunch:fastai.vision.data.ImageDataBunch=None,
                               ds_type:fastai.basic_data.DatasetType=None,
                               tta:bool=False,                                          
                               threshold = 0.5,                              
                               scale:float = 1.35,
                               beta: float = 0.4):
    """
    Option 1: Hand over a fastai.vision.learner and fastai.data_block.LabelList. No tta and no ensembling available
                for this option.
    Option 2: Hand over a fastai.vision.learner that was initalized with a fastai.vision.data.ImageDataBunch object.
    Option 3: Hand over dict where the keys are functions to create a model (e.g. torchvision.models.resnet50)
                and the values are paths to saved weights. Do this to use ensembling.
    
    Params:
        threshold:  threshold to consider the predictions per tile to be correct or not
        scale: only needed when tta is True; scale value for fastai's fastai.basic_train.Learner.TTA function
        beta: only needed when tta is True; beta value for fastai's fastai.basic_train.Learner.TTA function
    """
    
    if labelList is not None and ds_type is not None:
        raise ValueError('One of dataset or ds_type must be None')
    if labelList is not None and tta is True:
        raise ValueError('TTA is not available for a custom LabelList')
                
    #key:path, value:tuple (fastai.core.MultiCategory, tensor preds one hot encoded, tensor with pure preds) 
    #e.g. (MultiCategory 0, tensor([1., 0., 0., 0.]), tensor([0.9952, 0.0015, 0.0021, 0.0029]))
    path_to_pred = {}
    
    #Option 1
    if learner is not None and labelList is not None:
        for n, path in tqdm(enumerate(labelList.items), total=len(labelList.items)):
            pred = learner.predict(labelList[n][0], thresh=threshold)
            path_to_pred[path] = pred
    
    #Option 2
    elif learner is not None and labelList is None and  not dict_arch_to_path_of_saved_model and imageDataBunch is None:
        if tta is True:
            preds = learner.TTA(beta=beta, scale=scale, ds_type=ds_type)
        else:
            preds = learner.get_preds(ds_type=ds_type)
        path_to_pred = from_preds_to_dict_path_to_preds(preds, learner.data, ds_type, threshold)
                
    #Option 3
    elif dict_arch_to_path_of_saved_model and imageDataBunch is not None:
        preds = ensemble_predict(dict_arch_to_path_of_saved_model, imageDataBunch, ds_type, tta, scale, beta)
        path_to_pred = from_preds_to_dict_path_to_preds(preds, imageDataBunch, ds_type, threshold)                
               
    #key: id of a case; value: list with this syntax  
    #[<number of all tiles of one id>, 
    #[<number of occurence of predicted class1 over all tiles per id>, 
    #<number of occurence of predicted class2 over all tiles per id>, ..., 
    #<number of occurence of predicted classN over all tiles per id>],
    #y_true_one_hot_encoded]
    class_occurence_per_id = {}
    
    for path, pred in path_to_pred.items():   
        id = get_id_from_path(path)
        if id in class_occurence_per_id:
            v = class_occurence_per_id[id]
            v[0] = v[0] + 1
            v[1] = v[1] + pred[1]
            class_occurence_per_id[id] = v
        else:
            class_occurence_per_id[id] = [1, pred[1], one_hot_encode(label_func(path), lbs2num.values())]
            
    return class_occurence_per_id


def get_preds_threshold_per_id(thresholds_per_class:list, class_occurence_per_id:dict):
    """
    thresholds_per_class:  list with n = number_of_classes double values, e.g. four classes -> [0.5, 0.5, 0.5, 0.5] 
                            thresholds per class to consider the predictions per wsi to be correct or not
                            example: 100 tiles of one WSI, threshold for class 1 is 0.5, the wsi will get labeled with
                                        that class, if more than 50 tiles were labeled with that class by the classifier
                                
    RETURN:
            key: id of a case; 
            value: list with this syntax  
            [y_pred_th e.g. [True,False,False,False], 
            y_true e.g. [1,0,0,0]]
    """    
    result = {}
    for k in class_occurence_per_id.keys():
        y_pred_th = []
        for n, i in enumerate(class_occurence_per_id[k][1]):
            i = int(i)
            y_pred_th.append(i/class_occurence_per_id[k][0] > thresholds_per_class[n])
    
        result[k] = [y_pred_th, class_occurence_per_id[k][2]]
    return result

def get_accuracy_over_all_ids(number_of_ids, preds_threshold_per_id:dict, per_class:bool = True, number_of_classes = len(lbs2num)):
    if per_class is True:
        correctly_predicted = np.zeros(number_of_classes, dtype=np.int)
    else:
        correctly_predicted = 0
    for k in preds_threshold_per_id.keys():
        pred = preds_threshold_per_id[k][0]
        true = preds_threshold_per_id[k][1]
        for i in range(number_of_classes):
            if true[i] == pred[i]:
                if per_class is True:
                    correctly_predicted[i] = correctly_predicted[i] + 1
                else:
                    correctly_predicted = correctly_predicted + 1
    if per_class is True:                    
        correctly_predicted_percentage = {}
        for lb, num in zip(lbs2num.keys(), correctly_predicted):
            correctly_predicted_percentage[lb] = num/number_of_ids
    if per_class is False:
        correctly_predicted_percentage = correctly_predicted/number_of_ids

    return correctly_predicted_percentage

In [ ]:
arches = {resnext101_32x8d:Path(MODEL_PATH/'6-resnext101_32x8d-size512-bs8-seed_73/bestmodel_15'),
          se_resnext101_32x4d:MODEL_PATH/'11-se_resnext101_32x4d-size512-bs10-epochs_head5-epochs_complete5-seed_73/11-se_resnext101_32x4d-size512-bs8-epochs_head5-epochs_complete5-seed_73-complete'}

ths = [0.5,0.5,0.5,0.5]

## val set

In [ ]:
n_splits = 5
n = 13
saved_model_names = ['bestmodel_lr2=1e-7--lr3=1e-6_2nd_run_9', 'bestmodel_lr2=1e-7--lr3=1e-6_9', 'bestmodel_lr2=1e-7--lr3=1e-6_9', 'bestmodel_lr2=1e-7--lr3=1e-6_9', 'bestmodel_lr2=1e-7--lr3=1e-6_9']
copi_val = None
separated = False

In [ ]:
for iteration in tqdm(range(n_splits)):
    x_case_id_indices = list(range(len(case_ids)))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    splits = kf.split(x_case_id_indices)
    split_current_iteration = list(splits)[iteration]
    train_indices = split_current_iteration[0]
    val_indices = split_current_iteration[1]
    ids_train = [case_ids[i] for i in train_indices]
    ids_val = [case_ids[i] for i in val_indices]
    df_tile_paths_train_and_valid = pd.DataFrame(tile_paths_all, columns=['name'])
    
    tfms = get_transforms(flip_vert=True)

    df_c = pd.read_excel(LABELS_CORTICOTROP)
    
    nw = 16   #number of workers for data loader
    
    def split_func(path):
        path = Path(path)
        return get_id_from_path(path) in ids_val
    
    data = ImageList.from_df(df_tile_paths_train_and_valid, path=PATH)
    data = data.split_by_valid_func(split_func)
    data = data.label_from_func(label_func)
    data = data.transform(tfms=tfms, size=sz)
    temporary_training_path = PATH/f'models/{n}-resnext_currently_training_cross-valid-iteration-{iteration}'
    data = data.databunch(bs=bs, num_workers=nw, path=temporary_training_path)
    data = data.normalize()

    arch = resnext101_32x8d
    learner = cnn_learner(data=data, 
                         base_arch=arch, 
                         metrics=[accuracy_thresh], 
                         ps=dropout, 
                         pretrained=True, 
                         wd = wd)

    learner.load(saved_model_names[iteration])

    copi_val_current = get_class_occurence_per_id(learner=learner, ds_type=DatasetType.Valid, tta=False)

    if separated:
        if copi_val is None:
            copi_val = []
            copi_val.append(copi_val_current)
        else:
            copi_val.append(copi_val_current)
    else:
        if copi_val is None:
            copi_val = copi_val_current
        else:
            copi_val = {**copi_val, **copi_val_current}

In [ ]:
#copi_val_save_name = "13-5_fold_cross_validation-copi_val-separated=True.pkl"
copi_val_save_name = "13-5_fold_cross_validation-copi_val-separated=False.pkl"

In [ ]:
###
# save it to disc as pickle file
###
import pickle
f = open(f'{copi_val_save_name}',"wb")
pickle.dump(copi_val,f)
f.close()

In [ ]:
###
# load it from disc
###
f = open(copi_val_save_name,'rb')
copi_val = pickle.load(f)
f.close()

### thresholded accuracy per class

In [ ]:
if separated:
    copi_val_all = copi_val

In [ ]:
if separated:
    i=4
    copi_val = copi_val_all[i]

In [ ]:
preds_th_val = get_preds_threshold_per_id(ths, copi_val)

In [ ]:
accuracy_per_class_val = get_accuracy_over_all_ids(len(preds_th_val), preds_th_val)

In [ ]:
accuracy_per_class_val

### roc_curves and probability histograms

In [ ]:
class_names = ['acth', 'silent', 'lh', 'fsh']
class_numbers = [0,1,2,3]

In [ ]:
for class_name, class_number in zip(class_names, class_numbers):
    ###
    # roc_curve
    ###
    y_preds = []
    y_true = []
    for key, value in copi_val.items():
        #calculate prediction score for {class_name}
        n_tiles_all = value[0]
        n_tiles_class = value[1][class_number]
        percentage = n_tiles_class/n_tiles_all
        y_preds.append(percentage)
        
        #true label for {class_name}
        y_true.append(value[2][class_number])
        
    
    fpr, tpr, threshold = roc_curve(y_true, y_preds, pos_label=1)
    
    roc_auc = auc(fpr, tpr)
    
    plt.title(f'Receiver Operating Characteristic for {class_name}')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
    
    
    ###
    # histogram
    ###
    #predicted probabilities for {class_name} for WSIs, that were really {class_name}
    probs_true_positive = []
    #predicted probabilities for {class_name} for WSIs, that were really not {class_name}
    probs_true_negative =[]
    for key, value in copi_val.items():
        #calculate prediction score for {class_name}
        n_tiles_all = value[0]
        n_tiles_class = value[1][class_number]
        percentage = n_tiles_class/n_tiles_all
        if value[2][class_number] == 1:
            probs_true_positive.append(percentage)
        else:
            probs_true_negative.append(percentage)
    
    from matplotlib import pyplot
    
    bins = np.linspace(0, 1, 50)
    
    pyplot.hist(probs_true_positive, bins, alpha=0.5, label='true positive')
    pyplot.hist(probs_true_negative, bins, alpha=0.5, label='true negative')
    pyplot.legend(loc='upper right')
    pyplot.title(label=class_name)
    pyplot.ylabel('Frequency')
    pyplot.xlabel('Predicted Probability')
    pyplot.show()

In [ ]:
class_name = 'gonadotropic'
class_number_lh = 2 
class_number_fsh = 3

###
# roc_curve
###
y_preds = []
y_true = []
for key, value in tqdm(copi_val.items()):
    #calculate prediction score for {class_name}
    n_tiles_all = value[0]
    if value[1][class_number_lh] > value[1][class_number_fsh]:
        n_tiles_class = value[1][class_number_lh]
    else:
        n_tiles_class = value[1][class_number_fsh]

    percentage = n_tiles_class/n_tiles_all
    y_preds.append(percentage)
    
    #true label for {class_name}
    y_true.append(int(value[2][class_number_lh] == 1 or value[2][class_number_fsh] == 1))
    

fpr, tpr, threshold = roc_curve(y_true, y_preds, pos_label=1)

roc_auc = auc(fpr, tpr)

plt.title(f'Receiver Operating Characteristic for {class_name}')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


###
# histogram
###
#predicted probabilities for {class_name} for WSIs, that were really {class_name}
probs_true_positive = []
#predicted probabilities for {class_name} for WSIs, that were really not {class_name}
probs_true_negative =[]
for key, value in tqdm(copi_val.items()):
    #calculate prediction score for {class_name}
    n_tiles_all = value[0]
    if value[1][class_number_lh] > value[1][class_number_fsh]:
        n_tiles_class = value[1][class_number_lh]
    else:
        n_tiles_class = value[1][class_number_fsh]
    percentage = n_tiles_class/n_tiles_all
    if value[2][class_number_lh] == 1 or value[2][class_number_fsh] == 1:
        probs_true_positive.append(percentage)
    else:
        probs_true_negative.append(percentage)
   
from matplotlib import pyplot
   
bins = np.linspace(0, 1, 50)
   
pyplot.hist(probs_true_positive, bins, alpha=0.5, label='true positive')
pyplot.hist(probs_true_negative, bins, alpha=0.5, label='true negative')
pyplot.legend(loc='upper right')
pyplot.title(label=class_name)
pyplot.ylabel('Frequency')
pyplot.xlabel('Predicted Probability')
pyplot.show()

## test set

### seed 73

In [ ]:
#copi_test = get_class_occurence_per_id(dict_arch_to_path_of_saved_model=arches,
#                                      imageDataBunch=data,
#                                      ds_type=DatasetType.Test)
copi_test = get_class_occurence_per_id(learner=learner, ds_type=DatasetType.Test, tta=False)
preds_th_test = get_preds_threshold_per_id(ths, copi_test)
accuracy_per_class_test = get_accuracy_over_all_ids(len(preds_th_test), preds_th_test)

In [ ]:
accuracy_per_class_test

### seed 42

# Interpreter

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
def custom_confusion_matrix(self, slice_size:int=1):
        "Confusion matrix as an `np.ndarray`."
        x=torch.arange(0,self.data.c)
        if slice_size is None: cm = ((self.pred_class==x[:,None]) & (self.y_true==x[:,None,None])).sum(2)
        else:
            cm = torch.zeros(self.data.c, self.data.c, dtype=x.dtype)
            for i in range(0, self.y_true.shape[0], slice_size):
                #cm_slice = ((self.pred_class[i:i+slice_size]==x[:,None])
                            #& (self.y_true[i:i+slice_size]==x[:,None,None])).sum(2)
                cm_slice = ((self.pred_class[i:i+slice_size]==x[:,None])
                            & (self.y_true[i:i+slice_size]==(x[:,None,None]).float())).sum(2)
                torch.add(cm, cm_slice, out=cm)
        return to_np(cm)
    
fastai.train.ClassificationInterpretation.confusion_matrix = custom_confusion_matrix

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10)